In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import sys
import numpy as np
import pandas as pd
# from scipy.misc import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from tensorflow.keras.layers import BatchNormalization, Layer
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform


from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng
import pickle

In [ ]:
# returns three lists: anchor, positive, negative
def get_triple_batch(batch_size, train_data):
    """
    Create three lists of anchor images, positive images, negative images
    """
    n_classes, n_examples, w, h, d = train_data.shape
    

    rng = np.random.default_rng()

    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes,size=(batch_size,),replace=False)
    
    # initialize 2 empty arrays for the input image batch
    anchor = np.zeros((batch_size, w, h, d))

    positive = np.zeros((batch_size, w, h, d))

    negative = np.zeros((batch_size, w, h, d))

    
    for i in range(batch_size):
        category = categories[i]
        idx_1 = np.random.randint(0, n_examples)
        anchor[i,:,:,:] = train_data[category, idx_1].reshape(w, h, d)
        
        idx_pos = np.random.randint(0, n_examples)
        cat_pos = category
        positive[i,:,:,:] = train_data[cat_pos,idx_pos].reshape(w, h, d)

        idx_neg = np.random.randint(0, n_examples)
        cat_neg = (category + np.random.randint(1,n_classes)) % n_classes
        negative[i,:,:,:] = train_data[cat_neg,idx_neg].reshape(w, h, d)

    

    return anchor, positive, negative


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
train_in = open("mini-imagenet-cache-train.pkl", "rb")
train = pickle.load(train_in)
ungrouped_Xtrain = train["image_data"]
val_in = open("mini-imagenet-cache-val.pkl", "rb")
val = pickle.load(val_in)
ungrouped_Xval = val["image_data"]
val_data = ungrouped_Xval.reshape([16, 600, 84, 84, 3])

In [ ]:
train_mean = ungrouped_Xtrain.mean(axis=(0,1,2)) 
train_std = ungrouped_Xtrain.std(axis=(0,1,2))

In [ ]:
ungrouped_Xtrain = ungrouped_Xtrain.astype('float32')
ungrouped_Xval = ungrouped_Xval.astype('float32')

In [ ]:

ungrouped_Xtrain[..., 0] -= train_mean[0]
ungrouped_Xtrain[..., 1] -= train_mean[1]
ungrouped_Xtrain[..., 2] -= train_mean[2]
ungrouped_Xtrain[..., 0] /= train_std[0]
ungrouped_Xtrain[..., 1] /= train_std[1]
ungrouped_Xtrain[..., 2] /= train_std[2]


ungrouped_Xval[..., 0] -= train_mean[0]
ungrouped_Xval[..., 1] -= train_mean[1]
ungrouped_Xval[..., 2] -= train_mean[2]
ungrouped_Xval[..., 0] /= train_std[0]
ungrouped_Xval[..., 1] /= train_std[1]
ungrouped_Xval[..., 2] /= train_std[2]



In [ ]:
train_data = ungrouped_Xtrain.reshape([64, 600, 84, 84, 3])
val_data = ungrouped_Xval.reshape([16, 600, 84, 84, 3])

### Loading the validation images into tensors

### Saving the validation tensors on disk

In [ ]:
def initialize_weights(shape, name=None, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [ ]:
def initialize_bias(shape, name=None, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [ ]:
def L2_Norm(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,
      keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [ ]:
emb_size = 4096
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    pos_input = Input(input_shape)
    neg_input = Input(input_shape)
    anchor_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(128, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(512, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(emb_size, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the three images
    encoded_p = model(pos_input)
    encoded_n = model(neg_input)
    encoded_a = model(anchor_input)
    
    # # Add a customized layer to compute the eucl distances
    # # L2_dists = tuple: (positive distance, negative distance)
    # L2_distances = Lambda(triplet_dists)([encoded_p, encoded_n, encoded_a])


    embedded_layers = tf.keras.layers.concatenate([encoded_a, encoded_p, encoded_n], axis=1)
    
    # Connect the inputs with the outputs
    # the output is going to be the distance itself
    siamese_net = Model(inputs=[pos_input,neg_input, anchor_input],outputs=embedded_layers)
    
    # return the model
    return siamese_net

In [ ]:
model = get_siamese_model((84, 84, 3))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 84, 84, 3)]  0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 84, 84, 3)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 84, 84, 3)]  0           []                               
                                                                                                  
 sequential (Sequential)        (None, 4096)         23669376    ['input_1[0][0]',                
                                                                  'input_2[0][0]',            

In [ ]:
def fn_loss(margin=1):
    """Provides 'triplet loss' an enclosing scope with variable 'margin'.

  Arguments:
      margin: Integer, defines the baseline for distance for which pairs
              should be classified as dissimilar. - (default is 1).

  Returns:
      'triplet_loss' function with data ('margin') attached.
  """
    # L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    def triplet_loss(y_true, y_pred):
        anchor, positive, negative = y_pred[:,:emb_size], y_pred[:,emb_size:2*emb_size], y_pred[:,2*emb_size:]
        positive_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        negative_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
      
        return tf.reduce_mean(tf.maximum(positive_dist - negative_dist + margin, 0.))
    

    return triplet_loss

In [ ]:
# Hyper params
lr = 0.0001
margin = 1

In [ ]:
optimizer = Adam(learning_rate = lr)
model.compile(loss=fn_loss(margin=margin),optimizer=optimizer)

In [ ]:
def data_generator(data, batch_size):
    while True:
        anchor_input, pos_input, neg_input = get_triple_batch(batch_size, data)
        y = np.ones((batch_size))
        yield [pos_input,neg_input, anchor_input], y

In [ ]:
def val_generate(data, batch_size):
    anchor_input, pos_input, neg_input = get_triple_batch(batch_size, data)
    y = np.ones((batch_size))
    return [pos_input,neg_input, anchor_input], y

### Loading the train tensors

In [ ]:
# Hyper parameters
evaluate_every = 20 
# download_every = 1000
batch_size = 64
n_iter = 400 # No. of training iterations

# used for one batching testing
N_way = 10 # how many classes for testing one-shot tasks. has to be less than num classes in dataset
n_val = 250 # how many one-shot tasks to validate on

# used for straight validation testing
val_batch_size = 16

best = -1
epochs = 10
steps_per_epoch = 1000

In [ ]:
check_loss = fn_loss(margin=1)

In [ ]:
model_path = './weights/'
model_name = "triplet_loss_2"

In [ ]:
history = model.fit(
    data_generator(train_data, batch_size),
    steps_per_epoch=steps_per_epoch,
    validation_data = data_generator(val_data, val_batch_size),
    validation_steps= 10,
    epochs=epochs, verbose=True)

Epoch 1/10
1000/1000 [==============================] - 644s 636ms/step - loss: 1.2768 - val_loss: 1.0316
Epoch 2/10
1000/1000 [==============================] - 633s 633ms/step - loss: 0.9443 - val_loss: 0.9968
Epoch 3/10
1000/1000 [==============================] - 631s 631ms/step - loss: 0.8748 - val_loss: 0.8581
Epoch 4/10
1000/1000 [==============================] - 630s 630ms/step - loss: 0.8104 - val_loss: 0.8834
Epoch 5/10
1000/1000 [==============================] - 629s 629ms/step - loss: 0.7722 - val_loss: 0.8085
Epoch 6/10
1000/1000 [==============================] - 628s 628ms/step - loss: 0.7341 - val_loss: 0.8928
Epoch 7/10
1000/1000 [==============================] - 628s 628ms/step - loss: 0.7072 - val_loss: 0.8489
Epoch 8/10
1000/1000 [==============================] - 627s 627ms/step - loss: 0.6891 - val_loss: 0.7805
Epoch 9/10
1000/1000 [==============================] - 627s 627ms/step - loss: 0.6599 - val_loss: 0.7316
Epoch 10/10
1000/1000 [=======================

In [ ]:
epochs = 10
history = model.fit(
    data_generator(train_data, batch_size),
    steps_per_epoch=steps_per_epoch,
    validation_data = data_generator(val_data, val_batch_size),
    validation_steps= 10,
    epochs=epochs, verbose=True)

Epoch 1/10
1000/1000 [==============================] - 769s 769ms/step - loss: 0.6135 - val_loss: 0.8527
Epoch 2/10
1000/1000 [==============================] - 767s 767ms/step - loss: 0.5990 - val_loss: 0.7040
Epoch 3/10
1000/1000 [==============================] - 765s 765ms/step - loss: 0.5859 - val_loss: 0.7432
Epoch 4/10
1000/1000 [==============================] - 765s 765ms/step - loss: 0.5629 - val_loss: 0.5760
Epoch 5/10
1000/1000 [==============================] - 765s 765ms/step - loss: 0.5520 - val_loss: 0.7734
Epoch 6/10
1000/1000 [==============================] - 765s 765ms/step - loss: 0.5393 - val_loss: 0.7976
Epoch 7/10
1000/1000 [==============================] - 765s 765ms/step - loss: 0.5248 - val_loss: 0.8128
Epoch 8/10
1000/1000 [==============================] - 765s 765ms/step - loss: 0.5123 - val_loss: 0.6703
Epoch 9/10
1000/1000 [==============================] - 765s 765ms/step - loss: 0.5005 - val_loss: 0.8467
Epoch 10/10
1000/1000 [=======================

In [ ]:
model.save(os.path.join(model_path, 'weights_{}.h5'.format(model_name)))